In [8]:
import os
import numpy as np
import torch
import pandas as pd
import yaml

def one_hot(i):
    a=[0 for ii in range(7)]
    a[i]=1
    return a

def get_data_dev():
    label_transform={'neutral': 0, 'surprise': 1, 'fear': 2, 'sadness': 3, 'joy': 4, 'disgust': 5, 'anger': 6}
    data_dict = {}
    csv_path_dev = 'C:/Users/faustineljc/Desktop/mfcc_preprocess/dev_mfcc_d'
    csv_files_dev = os.listdir(csv_path_dev)
    csv_files_dev.sort(key=lambda x: (int(x.split("_")[0][3:]), int(x.split('_')[1][3:-4])))
    for i in range(len(csv_files_dev)):
        csv_files_dev[i] = csv_files_dev[i][:-4]
    with open('C:/Users/faustineljc/Desktop/melddatasets.yaml', 'rb') as stream:
        try:
            output = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    for file in csv_files_dev:
        for dtt in ['dev', 'test', 'train']:
            if file in output[dtt]:
                x, y = file, output[dtt][file]["Emotion"]
                data_dict[x] = one_hot(label_transform[y])
                break
    tmp_feature = {}
    for csv_file in csv_files_dev:
        data = pd.read_csv(csv_path_dev + '/' + csv_file + '.csv')
        feature = np.array(data, dtype='float32')[:, 2:].tolist()
        tmp_feature[csv_file] = feature
    return data_dict, tmp_feature

def get_data_test():
    label_transform = {'neutral': 0, 'surprise': 1, 'fear': 2, 'sadness': 3, 'joy': 4, 'disgust': 5, 'anger': 6}
    data_dict = {}
    csv_path_test = 'C:/Users/faustineljc/Desktop/mfcc_preprocess/test_mfcc_d'
    csv_files_test = os.listdir(csv_path_test)
    csv_files_test.sort(key=lambda x: (int(x.split("_")[0][3:]), int(x.split('_')[1][3:-4])))
    for i in range(len(csv_files_test)):
        csv_files_test[i] = csv_files_test[i][:-4]
    with open('C:/Users/faustineljc/Desktop/melddatasets.yaml', 'rb') as stream:
        try:
            output = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    for file in csv_files_test:
        for dtt in ['dev', 'test', 'train']:
            if file in output[dtt]:
                x, y = file, output[dtt][file]["Emotion"]
                data_dict[x] = one_hot(label_transform[y])
                break
    tmp_feature = {}
    for csv_file in csv_files_test:
        data = pd.read_csv(csv_path_test + '/' + csv_file + '.csv')
        feature = np.array(data, dtype="float32")[:, 2:].tolist()
        tmp_feature[csv_file] = feature
    return data_dict, tmp_feature

def get_data_train():
    label_transform = {'neutral': 0, 'surprise': 1, 'fear': 2, 'sadness': 3, 'joy': 4, 'disgust': 5, 'anger': 6}
    data_dict = {}
    csv_path_train ='C:/Users/faustineljc/Desktop/mfcc_preprocess/train_mfcc_d'
    csv_files_train = os.listdir(csv_path_train)
    #print(csv_files_train)
    csv_files_train.sort(key=lambda x:(int(x.split("_")[0][3:]), int(x.split('_')[1][3:-4])))
    for i in range(len(csv_files_train)):
        csv_files_train[i]=csv_files_train[i][:-4]
    with open('C:/Users/faustineljc/Desktop/melddatasets.yaml', 'rb') as stream:
        try:
            # print(yaml.safe_load(stream))
            output = yaml.safe_load(stream)
            # print(output)
            # print(len(output), type(output))  # len 3, type dict
            # print(output.keys())  # dict_keys(['dev', 'test', 'train'])
            # dev:1108 samples , test:2610 samples , train:9989 samples
            # print(len(output["dev"]),len(output["test"]),len(output["train"]))
            # print(output["dev"])
        except yaml.YAMLError as exc:
            print(exc)
    for file in csv_files_train:
        for dtt in ['dev', 'test', 'train']:
            if file in output[dtt]:
                x, y=file, output[dtt][file]["Emotion"]
                data_dict[x] = one_hot(label_transform[y])
                break
    tmp_feature = {}
    for csv_file in csv_files_train:
        data = pd.read_csv(csv_path_train + '/'+ csv_file + '.csv')
        feature = np.array(data, dtype= "float32")[:,2:].tolist()
        tmp_feature[csv_file]=feature
    # audio_feature = torch.tensor(tmp_feature)  # (n,num_time_step,39)
    # print(audio_feature.shape)
    # #print(audio_feature)
    # print(csv_files_train[:15])
    return data_dict, tmp_feature

In [ ]:
# it takes times
xy_train=get_data_train()  # a tuple (dict, dict)
#with open("xy_train.txt", "w") as f:
#    f.write(xy_train)
xy_dev=get_data_dev()
#with open("xy_dev.txt", "w") as f:
#    f.write(xy_dev)
xy_test=get_data_test()
#with open("xy_test.txt", "w") as f:
#    f.write(xy_test)

In [3]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
class Mydata_dev(Dataset):
    def __init__(self):
        super().__init__()
        self.xy_dev=xy_dev
    def __getitem__(self, index):
        file_name_dev = list(self.xy_dev[0].keys())[index]
        # data, emotion pair
        #nor = (torch.Tensor(self.xy_dev[1][file_name_dev])-torch.mean(torch.Tensor(self.xy_dev[1][file_name_dev]),dim=1,keepdim=True))/torch.std(torch.Tensor(self.xy_dev[1][file_name_dev]),dim=1,keepdim=True)
        nor = torch.Tensor(self.xy_dev[1][file_name_dev])
        sample_dev = nor, torch.Tensor(self.xy_dev[0][file_name_dev])
        return sample_dev
    def __len__(self):
        return len(self.xy_dev[0])

class Mydata_train(Dataset):
    def __init__(self):
        super().__init__()
        self.xy_train=xy_train  # a dict
    def __getitem__(self, index):
        file_name_train=list(self.xy_train[0].keys())[index]
        # data, emotion pair
        #nor = (torch.Tensor(self.xy_train[1][file_name_train])-torch.mean(torch.Tensor(self.xy_train[1][file_name_train]),dim=1,keepdim=True))/torch.std(torch.Tensor(self.xy_train[1][file_name_train]),dim=1,keepdim=True)
        nor = torch.Tensor(self.xy_train[1][file_name_train])
        sample_train = nor, torch.Tensor(self.xy_train[0][file_name_train])
        return sample_train
    def __len__(self):
        return len(self.xy_train[0])

class Mydata_test(Dataset):
    def __init__(self):
        super().__init__()
        self.xy_test=xy_test
    def __getitem__(self, index):
        file_name_test = list(self.xy_test[0].keys())[index]
        # data, emotion pair
        #nor = (torch.Tensor(self.xy_test[1][file_name_test])-torch.mean(torch.Tensor(self.xy_test[1][file_name_test]),dim=1,keepdim=True))/torch.std(torch.Tensor(self.xy_test[1][file_name_test]),dim=1,keepdim=True)
        nor = torch.Tensor(self.xy_test[1][file_name_test])
        sample_test = nor, torch.Tensor(self.xy_test[0][file_name_test])
        return sample_test
    def __len__(self):
        return len(self.xy_test[0])
import random
def collate_batch(batch):
    label_list, text_list, = [], []
    for (_text, _label) in batch:
        label_list.append(_label.tolist())
        text_list.append(_text)
    #print(label_list)
    label_list = torch.Tensor(label_list)
    text_list = torch.nn.utils.rnn.pad_sequence(text_list, batch_first=True, padding_value=0)
    return text_list.to(device), label_list.to(device)

In [ ]:
# Hyper-parameters
num_classes = 7
num_epochs = 2
batch_size = 8
learning_rate = 0.001

input_size = 39
sequence_length = 28
hidden_size = 128
num_layers = 1

In [ ]:
d_train=Mydata_train()
d_dev=Mydata_dev()
d_test=Mydata_test()

print(len(d_train))
print(len(d_dev))
print(len(d_test))
# Data loader
train_loader = DataLoader(dataset=d_train,batch_size=batch_size,collate_fn=collate_batch,shuffle=True)
dev_loader = DataLoader(dataset=d_dev,batch_size=batch_size,collate_fn=collate_batch,shuffle=True)
test_loader = DataLoader(dataset=d_test,batch_size=batch_size,collate_fn=collate_batch,shuffle=False)

In [ ]:
iterator_train=iter(train_loader)
iterator_dev=iter(dev_loader)
iterator_test=iter(test_loader)
n_train=next(iterator_train)
n_dev=next(iterator_dev)
n_test=next(iterator_test)
print(n_train[0][-1,[0],:], n_train[1])
print(n_dev[0].shape, n_dev[1])
print(n_test[0].shape, n_test[1])

In [5]:
# single or multiple layer RNN
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, bidirectional=False)
        # -> x needs to be: (batch_size, seq, input_size)

        # or:
        # self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        # self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        #self.bn = nn.BatchNorm1d(2*hidden_size)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # x: (n, 28, 28), h0: (2, n, 128)

        # Forward propagate RNN
        out, _ = self.rnn(x, h0)
        # or:
        # out, _ = self.lstm(x, (h0,c0))

        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)

        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
        
        out = self.fc(out)
        # out: (n, 10)
        return out

In [5]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        # -> x needs to be: (batch_size, seq, input_size)
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, bidirectional=False)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # x: (n, 28, 28), h0: (2, n, 128)
        # Forward propagate GRU
        out, _ = self.gru(x, h0)
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
        out = self.fc(out)
        # out: (n, 10)
        return out
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        # -> x needs to be: (batch_size, seq, input_size)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=False)
        self.bn = nn.BatchNorm1d(hidden_size)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        #h0 = torch.normal(0,0.01,(2*self.num_layers, x.size(0), self.hidden_size)).to(device)
        #c0 = torch.normal(0,0.01,(2*self.num_layers, x.size(0), self.hidden_size)).to(device)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # x: (n, 28, 28), h0: (2, n, 128)
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0,c0))
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
        out = self.bn(out)
        out = self.fc(out)
        # out: (n, 7)
        return out

In [6]:
from torch.utils.data.sampler import WeightedRandomSampler
# Hyper-parameters
num_classes = 7
num_epochs = 10
batch_size = 8
learning_rate = 0.0001 #learning_rate = 0.00001

input_size = 39
hidden_size = 64
num_layers = 1

d_train=Mydata_train()
d_dev=Mydata_dev()
d_test=Mydata_test()


'''
balance data
methods for dealing with imbalanced datas： 1.oversampling 2.class weight 3.loss 

'''
#weights = torch.Tensor([4.0, 15.0, 15.0, 3.0, 1.0, 6.0, 3.0])/10
#class_weights_train = torch.Tensor([ 2.1208,  8.2896, 37.2724, 14.6252,  5.7309, 36.8598,  9.0072])
class_weights_train = torch.Tensor([4.0, 15.0, 15.0, 3.0, 1.0, 6.0, 3.0])
sample_weights = [0] * len(d_train)
for idx, (data, label) in enumerate(d_train):
    class_weight = class_weights_train[torch.argmax(label)]
    sample_weights[idx] = class_weight
sampler_train=WeightedRandomSampler(sample_weights,num_samples=len(d_train),replacement=True)

#class_weights_dev = torch.Tensor([ 2.3596,  7.3933, 27.7250,  9.9910,  6.8037, 50.4091,  7.2484])
class_weights_dev = torch.Tensor([4.0, 15.0, 15.0, 3.0, 1.0, 6.0, 3.0])
sample_weights = [0] * len(d_dev)
for idx, (data, label) in enumerate(d_dev):
    class_weight = class_weights_dev[torch.argmax(label)]
    sample_weights[idx] = class_weight
sampler_dev=WeightedRandomSampler(sample_weights,num_samples=len(d_dev),replacement=True)

#class_weights_test = torch.Tensor([ 2.0820,  9.3060, 52.3000, 12.5721,  6.5050, 38.4559,  7.5797])
class_weights_test = torch.Tensor([4.0, 15.0, 15.0, 3.0, 1.0, 6.0, 3.0])
sample_weights = [0] * len(d_test)
for idx, (data, label) in enumerate(d_test):
    class_weight = class_weights_test[torch.argmax(label)]
    sample_weights[idx] = class_weight
sampler_test=WeightedRandomSampler(sample_weights,num_samples=len(d_test),replacement=True)

# Data loader
train_loader = DataLoader(dataset=d_train,batch_size=batch_size,collate_fn=collate_batch,sampler = sampler_train)
dev_loader = DataLoader(dataset=d_dev,batch_size=batch_size,collate_fn=collate_batch,sampler = sampler_dev)
test_loader = DataLoader(dataset=d_test,batch_size=batch_size,collate_fn=collate_batch,sampler = sampler_test)

model = LSTM(input_size, hidden_size, num_layers, num_classes).to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

NameError: name 'xy_train' is not defined

In [19]:
# Train the model
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (mfccdata, labels) in enumerate(train_loader):
        # origin shape: [N, NN, 39]
        # resized: [N, 28, 28]
        # mfccdata = mfccdata.reshape(-1, sequence_length, input_size).to(device)
        mfccdata = mfccdata.to(device)
        #print(labels)
        labels = torch.argmax(labels,dim=1).to(device)

        # Forward pass
        outputs = model(mfccdata)
        loss = criterion(outputs, labels)
        if torch.any(torch.isnan(mfccdata)) or torch.any(torch.isnan(labels)) or torch.any(torch.isnan(outputs)):
            continue
        #print(labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i + 1) % 100 == 0 or i == 0:
            with torch.no_grad():
                n_correct = 0
                n_samples = 0
                for mfccdata, labels in dev_loader:
                    mfccdata = mfccdata.to(device)
                    labels = torch.argmax(labels,dim=1).to(device)
                    outputs = model(mfccdata)
                    # max returns (value ,index)
                    predicted = torch.argmax(outputs.data, 1).flatten()
                    #print("predicted:",predicted)
                    #print("labels   :",labels)
                    n_samples += len(labels)
                    n_correct += (predicted == labels).sum().item()

                acc = n_correct / n_samples
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{n_total_steps}], Loss: {loss.item():.4f}, Val_acc: {acc}')


In [20]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for mfccdata, labels in test_loader:
        mfccdata = mfccdata.to(device)
        labels = torch.argmax(labels,dim=1).to(device)
        outputs = model(mfccdata)
        predicted = torch.argmax(outputs.data, 1).flatten()
        #print("predicted:",predicted)
        #print("labels   :",labels)
        n_samples += len(labels)
        n_correct += (predicted == labels).sum().item()

    acc = n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')


In [ ]:
#methods for dealing with imbalanced datasets: 
#weighted random sampler 
#focal loss 不平衡文本分类，focal loss理论及pytorch实现 https://blog.51cto.com/u_15127571/3869386 



